In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math
import tiktoken 


ModuleNotFoundError: No module named 'tiktoken'

In [3]:
torch.manual_seed(1337)

In [4]:
string = "I went to the shops today to buy a back of milk"

enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode(string)
tokens = torch.tensor(tokens)

x = tokens[:-1]
y = tokens[1:]

NameError: name 'tiktoken' is not defined

In [9]:
B = 1      # batches    
T = 2       # context length    
C = 5      #  m_embd   
n_head = 1

linear = nn.Linear(C, 3*C)
def c_attn(x):
    return linear(x)

x = torch.randn(B, T, C)
print(f"x: {x}")

q, k, v = c_attn(x).split(C, dim=2)   #B, T, C
print(f"q{q}")

q = q.view(B, T, n_head, C // n_head)
print(q)
print(q.shape)

q = q.view(B, T, n_head, C // n_head).transpose(1, 2)    # B, nh, T, hs, # split embed dim into multiple heads (head size)
k = k.view(B, T, n_head, C // n_head).transpose(1, 2)    # B, nh, T, hs 
v = v.view(B, T, n_head, C // n_head).transpose(1, 2)    # B, nh, T, hs 
print(f"q:{q}")
print(q.shape)
print(f"k: {k}")
print(k.shape)
 
att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))

att = att * v


x: tensor([[[ 0.0942, -0.8734, -0.2254,  1.8452,  1.6645],
         [-0.2450, -1.5325,  1.1101, -0.6680, -0.3673]]])
qtensor([[[ 0.7546, -0.5453,  0.5953,  0.4464,  0.5580],
         [ 0.6405,  0.6410,  0.6809, -1.1272,  0.2000]]],
       grad_fn=<SplitBackward0>)
tensor([[[[ 0.7546, -0.5453,  0.5953,  0.4464,  0.5580]],

         [[ 0.6405,  0.6410,  0.6809, -1.1272,  0.2000]]]],
       grad_fn=<ViewBackward0>)
torch.Size([1, 2, 1, 5])
q:tensor([[[[ 0.7546, -0.5453,  0.5953,  0.4464,  0.5580],
          [ 0.6405,  0.6410,  0.6809, -1.1272,  0.2000]]]],
       grad_fn=<TransposeBackward0>)
torch.Size([1, 1, 2, 5])
k: tensor([[[[ 0.6260,  0.9029,  0.3953,  0.6671,  0.7866],
          [ 0.2280, -0.5402, -0.7216,  0.2327,  0.1444]]]],
       grad_fn=<TransposeBackward0>)
torch.Size([1, 1, 2, 5])


RuntimeError: The size of tensor a (2) must match the size of tensor b (5) at non-singleton dimension 3

In [36]:
B = 1   # batches
T = 8  # context length
C = 2   #  m_embd

x = torch.randn(B, T, C)        # B, T, C
tril = torch.tril(torch.ones((T, T)))   # T, T
wei = torch.zeros((T, T))             # dummy weights
wei = wei.masked_fill(tril== 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
print(wei)
out = wei @ x
print(out.shape)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])
torch.Size([1, 8, 2])


In [ ]:
dropout = 0.2


class CausalAttention:
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0       # what does this do
        # q, k, v for all heads in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_head)   # (n_embd, 3 * n_head) 3 for q, k, v
        # out 
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)       # (n_emnd, n_embd) # what does this do

        # regularisation 
        self.attn_dropout = nn.Dropout(dropout)
        self.resid_dropout = nn.Dropout(dropout)

        # causal mask -> only attend to previous 
        self.q = query
        self.k = k 
        self.v = value 

        # dot product queries and keys
        att = self.q * K.transpose(-1)  # which dimension transposing 
        
        # * dk**-0.5
        # softmax
        # * v


    # masking to prevent looking at forward tokens 
